In [ ]:
# Latitude,Longitude

def coordinates(city):
    
    import pandas as pd
    import requests
    from bs4 import BeautifulSoup

    url = "https://en.wikipedia.org/wiki/"

    city = requests.get(url+city)

    city = BeautifulSoup(city.text)

    for item in city.find_all("span", {"class":"geo"}):
        temp = item.text
        break

    flag = temp.split(";")
    return flag[0],flag[1]